In [16]:
pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 83.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 8.9 MB/s eta 0:00:0000:010:00:01
Note: you may need to restart the kernel to use updated packages.


In [18]:
import os
# import PyPDF2
import pdfplumber
import tqdm
import json
from sentence_transformers import SentenceTransformer

In [25]:
def segment_text(text):
    currLen = 0
    chunks = []
    currText = ""
    words = text.split()
    for word in words:
        if currLen + len(word) + (1 if currText else 0) <= 1024:
            currText += " "+word
            currLen += len(word) + 1
        else:

            chunks.append(currText)
            currLen = len(word)
            currText = word
    if currText:
        chunks.append(currText)
    return chunks

In [20]:
def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        # with open(pdf_path, 'rb') as file:
        #     reader = PyPDF2.PdfReader(file)
        #     for page_num in range(len(reader.pages)):
        #         page = reader.pages[page_num]
        #         text += page.extract_text() + "\n"
        with pdfplumber.open(pdf_path) as pdf:
            text = "\n".join([page.extract_text() for page in pdf.pages if page.extract_text()])
    except Exception as e:
        print(f"Error extracting text from {pdf_path}: {str(e)}")
    return text

In [21]:
def getData(dirPath):
  dir = dirPath
  text_dict = {}
  for file in tqdm.tqdm_notebook(os.listdir(dir), total = len(os.listdir(dir)), desc="Extracting Text"):
      if file.endswith(".pdf"):
          pdf_path = os.path.join(dir, file)
          # print(pdf_path)
          text = extract_text_from_pdf(pdf_path)
          text_dict[file] = text
  o_chunks = 0
  content = {}
  for key, value in tqdm.tqdm_notebook(text_dict.items(), total=len(text_dict), desc="Making the Final Json"):
      textChunks = segment_text(value)
      for i, chunk in enumerate(textChunks):
          content[f"chunk-{o_chunks}"] = {
              "interChunkId": i,
              "fileName": key,
              "text": chunk
          }
          o_chunks += 1
  return content


In [22]:
def findEmbeddings(content):
  model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
  allRelations = []
  for key, value in content.items():
      allRelations.append(value["text"])
  embeddings = model.encode(allRelations)
  for key, embed in zip(content.keys(), embeddings):
      content[key]["embedding"] = embed.tolist()
  return content

In [23]:
domain_Dirs_path = "/kaggle/input/researchpapercorpus"
domain_Dirs = os.listdir(domain_Dirs_path)
domain_Dirs

['Biology_', 'Law_', 'Education_', 'Finance_']

In [24]:
for domain in domain_Dirs:
    if domain == "Law_":
        continue
    print("Processing the files for the domain: ", domain)
    inputFolder = domain_Dirs_path+"/"+domain+"/"
    content = getData(inputFolder)
    print("Got the data for the domain: ", domain, " total chunks: ", len(content))
    writeFile = f"/kaggle/working/{domain}Full.json" 
    with open(writeFile, 'w', encoding="utf-8") as f:
        json.dump(content, f, indent = 4)

Processing the files for the domain:  Biology_


<ipython-input-21-509ebacc1626>:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for file in tqdm.tqdm_notebook(os.listdir(dir), total = len(os.listdir(dir)), desc="Extracting Text"):


Extracting Text:   0%|          | 0/96 [00:00<?, ?it/s]

Error extracting text from /kaggle/input/researchpapercorpus/Biology_/q-bio_Symmetry_and_Dynamics_in_living_organisms_The_self-similarity_principle_governs_gene_expression_dynamics.pdf: No /Root object! - Is this really a PDF?
Error extracting text from /kaggle/input/researchpapercorpus/Biology_/q-bio_Vibrational_energy_relaxation_in_proteins.pdf: No /Root object! - Is this really a PDF?
Error extracting text from /kaggle/input/researchpapercorpus/Biology_/q-bio_Molecular_Dynamics_Simulations_of_a_Nucleosome_and_Free_DNA.pdf: No /Root object! - Is this really a PDF?
Error extracting text from /kaggle/input/researchpapercorpus/Biology_/q-bio_Amino_acid_substitution_matrices_for_protein_conformation_identification.pdf: No /Root object! - Is this really a PDF?
Error extracting text from /kaggle/input/researchpapercorpus/Biology_/q-bio_Study_of_a_model_for_the_folding_of_a_small_protein.pdf: No /Root object! - Is this really a PDF?
Error extracting text from /kaggle/input/researchpapercorp

<ipython-input-21-509ebacc1626>:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for key, value in tqdm.tqdm_notebook(text_dict.items(), total=len(text_dict), desc="Making the Final Json"):


...

Got the data for the domain:  Education_  total chunks:  132
Processing the files for the domain:  Finance_


Extracting Text:   0%|          | 0/99 [00:00<?, ?it/s]

Error extracting text from /kaggle/input/researchpapercorpus/Finance_/q-fin_IMEX_schemes_for_a_Parabolic-ODE_system_of_European_Options_with_Liquidity_Shocks.pdf: No /Root object! - Is this really a PDF?
Error extracting text from /kaggle/input/researchpapercorpus/Finance_/q-fin_Bermudan_options_by_simulation.pdf: No /Root object! - Is this really a PDF?
Error extracting text from /kaggle/input/researchpapercorpus/Finance_/q-fin_High_Performance_Financial_Simulation_Using_Randomized_Quasi-Monte_Carlo_Methods.pdf: No /Root object! - Is this really a PDF?
Error extracting text from /kaggle/input/researchpapercorpus/Finance_/q-fin_A_hybrid_treefinite-difference_approach_for_Heston-Hull-White_type_models.pdf: No /Root object! - Is this really a PDF?
Error extracting text from /kaggle/input/researchpapercorpus/Finance_/q-fin_Optimal_decision_for_the_market_graph_identification_problem_in_sign_similarity_network.pdf: No /Root object! - Is this really a PDF?
Error extracting text from /kaggle

Making the Final Json:   0%|          | 0/99 [00:00<?, ?it/s]

Got the data for the domain:  Finance_  total chunks:  0


In [34]:
with open("/kaggle/working/Law_Full.json", 'rb') as f:
    data1 = json.load(f)
for da in data1:
    if len(data1[da]['text']) > 1024:
        print(len(data1[da]['text']))